# Modularity and community extraction for all matices files
#### Quantifys modularity and plots topology maps and gemomentric maps based on communities 

In [ ]:
pip install python-louvain

In [ ]:
import os
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt
from matplotlib.colors import ColorConverter
import seaborn as sns
import random
import numpy as np
import networkx as nx
import pandas as pd
from networkx.algorithms.community import greedy_modularity_communities


# Directory of Excel files
input_dir = r"D:\MEA_Analysis\.....\21_02_Thresholded_Fils_Concatinated"
output_dir = r"D:\......\ModularityExcel_Res_x.x"
# Define the output directory
output_dir_plots = r"D:\........\ModularityPlots_Res_x.x"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_plots, exist_ok=True)


# Iterate over each Excel file in the directory
for excel_file in os.listdir(input_dir):
    if excel_file.endswith('.xlsx'):
        file_path = os.path.join(input_dir, excel_file)
        xls = pd.ExcelFile(file_path)
        
        # Remove the '.xlsx' extension and extract the last 3 digits before .xlsx
        file_prefix = excel_file[:-5][-3:]
        # Create new prefix by adding 'Thre_' to the extracted part
        new_file_prefix = f'Ther_{file_prefix}_'

        # Initialize a list to store results for each sheet
        results = []
        # Initialize a dictionary to store community DataFrames for each sheet
        community_dfs = {}
        
        # Iterate over each sheet (sample) in the Excel file
        for sheet_name in xls.sheet_names:
            data = pd.read_excel(xls, sheet_name=sheet_name)
            neuron_names = data.iloc[:, 0].values

            # Sort correlation matrix based on neuron_names order
            correlation_matrix = data.iloc[:, 5:].copy()
            correlation_matrix.columns = neuron_names
            correlation_matrix.index = neuron_names
            correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
            correlation_matrix = correlation_matrix.reindex(neuron_names)
            np.fill_diagonal(correlation_matrix.values, 0)

            
            ######## Use escel file name extracted Threshold value  to set threshold for weighted-G #####
            
            # Convert the extracted string to a float
            threshold_value = float(file_prefix)
            
            
            ############################# Create a weighted graph  ##########################
            G = nx.Graph()
            for i, neuron_i in enumerate(neuron_names):
                for j, neuron_j in enumerate(neuron_names):
                    if i < j:
                        correlation_coefficient = correlation_matrix.iloc[i, j]
                        if correlation_coefficient > 0.1:  # Set threshold 
                            G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

            
            ############################# Modularity and community detection ####################
            
            # Community detection using Louvain method
        
            partition = community_louvain.best_partition(G, resolution=1.0)    ## define resolutioin
            
            community_list = [[] for _ in range(max(partition.values()) + 1)]
            for neuron, community_id in partition.items():
                community_list[community_id].append(neuron)
            
            # Calculate modularity
            Q = nx.community.modularity(G, community_list)
            num_communities = len(community_list)
            avg_community_size = np.mean([len(community) for community in community_list])
            
            # Append results for this sheet to the list
            results.append([excel_file, sheet_name, Q, num_communities, avg_community_size])

            print("Modularity (Q) =", Q)
            print(community_list)    

            # Extract x and y positions of nodes   ## was not used in this code
            x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
            y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

            # Calculate node degrees
            node_degrees = dict(G.degree())

            # Define community colors
            community_colors = [f"C{idx}" for idx in range(len(community_list))]

            # Define lighter colors for communities
            lighter_community_colors = []
            for color in community_colors:
                rgb_color = ColorConverter().to_rgb(color)
                lighter_color = tuple([min(1, c + 0.3) for c in rgb_color])  # Adjust the increment (0.3) as needed
                lighter_community_colors.append(lighter_color)

            
            
            ###################################### Plot Geoometric Connectivity Maps ######################################   

            # Plot communities based on real position
            plt.figure(figsize=(12, 5))

            # Plot based on real position
            plt.subplot(1, 2, 1)

            plt.gca().set_aspect('equal')  # Set aspect ratio to be equal

            community_colors = [f"C{idx}" for idx in range(len(community_list))]  # Generate community colors
            for source, target in G.edges():
                pos_src = (x_positions[neuron_names == source][0], y_positions[neuron_names == source][0])
                pos_tgt = (x_positions[neuron_names == target][0], y_positions[neuron_names == target][0])
                plt.annotate("",
                             xy=pos_src, xycoords='data',
                             xytext=pos_tgt, textcoords='data',
                             arrowprops=dict(arrowstyle="-", color='gray', alpha=0.4, linewidth=0.3,
                                             connectionstyle="arc3,rad=0.2", zorder=1))  # Adjust the curvature with rad parameter

            for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
                community_x = x_positions[np.isin(neuron_names, community)]
                community_y = y_positions[np.isin(neuron_names, community)]

                # Add jitter to x and y positions
                jittered_community_x = [x + random.uniform(-30, 30) for x in community_x]
                jittered_community_y = [y + random.uniform(-30, 30) for y in community_y]

                community_degrees = [node_degrees[node] for node in community]
                node_size = [deg * 3 for deg in community_degrees]  # Adjust the multiplier as needed
                plt.scatter(jittered_community_x, jittered_community_y, label=f'Community {idx + 1}',
                            s=node_size, c=color, zorder=3, edgecolor='black', linewidth=0.3)

            print(excel_file, '-------------------')
            plt.xlabel('X Position')
            plt.ylabel('Y Position')
            plt.title(f'{sheet_name}_Network Community Visualization (Real Position)')
            plt.grid(True, color='gray', alpha=0.2, linewidth=0.5)  # Adjust grid lines
            plt.axis('equal')  # Ensure aspect ratio is equal
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            # Adjust the y-axis limits to -200 and 1600
            plt.ylim(-200, 1600)
            
            
            
            #### in case you want to extract only geometry plot ####
            # Use the new prefix in the filename for saving the plot
            #plot_filename = f'{new_file_prefix}{sheet_name}_NetComPlot'
            # Save the plot as PDF with the sheet name included
            #plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_Geometry.pdf'), format='pdf')
            # Save the plot as SVG
            #plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_Geometry.svg'), format='svg')


            ###################### Plot based on topology --- using Kamada-Kawai layout
            plt.subplot(1, 2, 2)
            pos = nx.kamada_kawai_layout(G, scale=2.0)  # Using Kamada-Kawai layout with increased scale for node separation
            for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
                community_degrees = [node_degrees[node] for node in community]
                node_size = [deg * 3 for deg in community_degrees]  # Adjust the multiplier as needed

                plt.scatter([pos[node][0] for node in community], [pos[node][1] for node in community], label=f'Community {idx + 1}',
                            s=node_size, c=color, zorder=3, edgecolor='black', linewidth=0.3)


            for source, target in G.edges():
                pos_src = pos[source]
                pos_tgt = pos[target]
                nx.draw_networkx_edges(G, pos, edgelist=[(source, target)], 
                                       width=0.2, alpha=0.9, connectionstyle="arc3,rad=0.2",
                                       edge_color='gray')  # Curved edges without arrowheads

            print(excel_file, '-------------------')
            plt.title(f'{sheet_name}_Network Community Visualization (Kamada-Kawai Layout)')

            plt.tight_layout()

            ########### Save Geometry-based  and Topology_based  plots together
            # Use the new prefix in the filename for saving the plot
            plot_filename = f'{new_file_prefix}{sheet_name}_NetComPlot'
            # Save the plot as PDF
            plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_Topology.pdf'), format='pdf')
            # Save the plot as SVG
            plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_Topology.svg'), format='svg')

            plt.show()



            ############################################ Plot community-based Hetmaps #################################

            # Print communities
            for idx, community in enumerate(community_list):
                print(f"Community {idx + 1}: {community}")

            # Extract ordered neurons  (neurons that are within communities will be selected)
            ordered_neurons = [neuron for community in community_list for neuron in community]

            # Create a symmetric correlation matrix with ordered neurons
            # first: generates list of node pairs with their correlation coefficient attributed. 
            # Then: it generates a symmetric matrix based on list 
            ordered_data = pd.DataFrame(index=ordered_neurons, columns=ordered_neurons)
            for i, neuron_i in enumerate(ordered_neurons):
                for j, neuron_j in enumerate(ordered_neurons):
                    correlation_coefficient = correlation_matrix.loc[neuron_i, neuron_j]
                    ordered_data.at[neuron_i, neuron_j] = correlation_coefficient
                    ordered_data.at[neuron_j, neuron_i] = correlation_coefficient


            # Ensure diagonal is zero
            np.fill_diagonal(ordered_data.values, 0)

            # Convert correlation coefficients to floats
            ordered_data = ordered_data.astype(float)

            # Plot the heatmap
            plt.figure(figsize=(7, 5))

            # Plot the heatmap with grayscale colormap
            sns.heatmap(ordered_data, cmap='Greys', vmin=ordered_data.values.max(), vmax=ordered_data.values.min(),
                        linewidths=0.5, linecolor='black', alpha=0.5, cbar=True, square=True, mask=ordered_data.isnull(),
                        annot=False, fmt='.2f')

            # Draw squares around neurons of each community with respective colors
            ax = plt.gca()
            start = 0
            for idx, community in enumerate(community_list):
                end = start + len(community)
                color = community_colors[idx]
                rect = plt.Rectangle((start, start), end - start, end - start, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
                ax.add_patch(rect)
                start = end

            print(excel_file, '-------------------')
            plt.title(f'{sheet_name}_Community-Based Correlation Matrix Heatmap')
            plt.xlabel('Neurons')
            plt.ylabel('Neurons')

            
            # Use the new prefix in the filename for saving the plot
            plot_filename = f'{new_file_prefix}{sheet_name}_NetComPlot'
            # Save the plot as PDF
            plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_HeatMap.pdf'), format='pdf')
            # Save the plot as SVG
            plt.savefig(os.path.join(output_dir_plots, f'{plot_filename}_HeatMap.svg'), format='svg')
            plt.show()
            
            ######################################################################################################
            ######################################################################################################
            ######## EXPORT Community data ##############
            
            # Export the list of neurons in each detected community to a separate DataFrame
            community_df = pd.DataFrame([(community_id, neuron) for community_id, community in enumerate(community_list) for neuron in community], columns=['Community', 'Neuron'])
            
            # Store community DataFrame for this sheet in the dictionary
            community_dfs[sheet_name] = community_df
            
            
            # Export the list of neurons in each detected community to a separate Excel file
            #community_df = pd.DataFrame([(community_id, neuron) for community_id, community in enumerate(community_list) for neuron in community], columns=['Community', 'Neuron'])
            #community_df.to_excel(os.path.join(output_dir, f"{excel_file}_{sheet_name}_communities.xlsx"), index=False)

        # Create a DataFrame from the results list
        results_df = pd.DataFrame(results, columns=['Source Excel File', 'Source Excel Sheet', 'Modularity (Q)', 'Number of Communities', 'Average Community Size'])
        
        # Export the results DataFrame to an Excel file
        results_df.to_excel(os.path.join(output_dir, f"{excel_file}_results.xlsx"), index=False)
        
        # Export community DataFrames to a single Excel file
        with pd.ExcelWriter(os.path.join(output_dir, f"{excel_file}_communities.xlsx")) as writer:
            for sheet_name, df in community_dfs.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            

# Modularity

- Whole network modularity score based on Louvain method

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
from networkx.algorithms.community import greedy_modularity_communities

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0:  # Consider only positive correlations
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Relabel the nodes
G = nx.relabel.relabel_nodes(G, dict(enumerate(correlation_matrix.columns)), copy=True)

# Community detection using Louvain method
communities = greedy_modularity_communities(G)
community_list = [list(community) for community in communities]

# Calculate modularity
Q = nx.community.modularity(G, community_list)

print("Modularity (Q) =", Q)
print (communities)
print(community_list)

- Modularity and communities based on Louvian -> greedy modularity

In [ ]:

## Consider that neurons that are outside of the communities will not be included in the communitys and will not be exported


import numpy as np
import networkx as nx
import pandas as pd
from networkx.algorithms.community import greedy_modularity_communities

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Replace NaN values with 0
correlation_matrix.fillna(0, inplace=True)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.0:  # Consider if threshold has not been applied before
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Relabel the nodes
G = nx.relabel.relabel_nodes(G, dict(enumerate(correlation_matrix.columns)), copy=True)

# Community detection using Louvain method
communities = greedy_modularity_communities(G)
community_list = [list(community) for community in communities]

# Create a mapping dictionary for node indices to neuron names
index_to_neuron = {i: neuron_names[i] for i in range(len(neuron_names))}

print("Length of neuron_names:", len(neuron_names))
print("Length of index_to_neuron:", len(index_to_neuron))

# Convert community lists from node indices to neuron names, excluding nodes not in index_to_neuron
neuron_community_list = [[index_to_neuron[node_index] for node_index in community if node_index in index_to_neuron] for community in community_list]

print("Length of neuron_names:", len(neuron_names))
print("Length of index_to_neuron:", len(index_to_neuron))
print("Number of communities:", len(community_list))


# Calculate modularity
Q = nx.community.modularity(G, community_list)

print("Modularity (Q) =", Q)
print("Communities:")
for i, community in enumerate(neuron_community_list):
    print(f"Community {i + 1}: {community}")

- Modularity and communities based on Louvian -> best_partition

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Threshold
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Print communities
for idx, community in enumerate(community_list):
    print(f"Community {idx + 1}: {community}")

Plot_communities_01

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Consider threshold
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Extract x and y positions of nodes
x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

# Plot communities based on real position
plt.figure(figsize=(12, 6))

# Plot based on real position
plt.subplot(1, 2, 1)
for idx, community in enumerate(community_list):
    community_x = x_positions[np.isin(neuron_names, community)]
    community_y = y_positions[np.isin(neuron_names, community)]
    plt.scatter(community_x, community_y, label=f'Community {idx + 1}')

    for source in community:
        for target in community:
            if source != target and G.has_edge(source, target):
                plt.plot([x_positions[neuron_names == source][0], x_positions[neuron_names == target][0]],
                         [y_positions[neuron_names == source][0], y_positions[neuron_names == target][0]],
                         color='gray', alpha=0.3, linewidth=2)  # Adjust the line thickness here

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Network Community Visualization (Real Position)')
plt.legend()
plt.grid(True)

# Plot based on topological closeness using Kamada-Kawai layout
plt.subplot(1, 2, 2)
pos = nx.kamada_kawai_layout(G)  # Using Kamada-Kawai layout
for idx, community in enumerate(community_list):
    nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=f"C{idx}", node_size=100)
nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.7)
plt.title('Network Community Visualization (Kamada-Kawai Layout)')

plt.tight_layout()
plt.show()

- Plot communities_02 (node size takes degree of node)

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Consider Threshold
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Extract x and y positions of nodes
x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

# Calculate node degrees
node_degrees = dict(G.degree())

# Plot communities based on real position
plt.figure(figsize=(12, 6))

# Plot based on real position
plt.subplot(1, 2, 1)
community_colors = [f"C{idx}" for idx in range(len(community_list))]  # Generate community colors
for idx, (community, color) in enumerate(zip(community_list, community_colors)):
    community_x = x_positions[np.isin(neuron_names, community)]
    community_y = y_positions[np.isin(neuron_names, community)]
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 10 for deg in community_degrees]  # Adjust the multiplier as needed
    plt.scatter(community_x, community_y, label=f'Community {idx + 1}', s=node_size, c=color)

    for source in community:
        for target in community:
            if source != target and G.has_edge(source, target):
                plt.plot([x_positions[neuron_names == source][0], x_positions[neuron_names == target][0]],
                         [y_positions[neuron_names == source][0], y_positions[neuron_names == target][0]],
                         color='gray', alpha=0.3, linewidth=1)  # Adjust the line thickness here

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Network Community Visualization (Real Position)')
plt.legend()
plt.grid(True, color='gray', alpha=0.2, linewidth=0.5)  # Adjust grid lines
plt.axis('equal')  # Ensure aspect ratio is equal

# Plot based on topological closeness using Kamada-Kawai layout
plt.subplot(1, 2, 2)
pos = nx.kamada_kawai_layout(G)  # Using Kamada-Kawai layout
for idx, (community, color) in enumerate(zip(community_list, community_colors)):
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 10 for deg in community_degrees]  # Adjust the multiplier as needed
    nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=color, node_size=node_size)
nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.7)
plt.title('Network Community Visualization (Kamada-Kawai Layout)')

plt.tight_layout()
plt.show()

- Plot Geometry_based and topology based

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Consider only positive correlations
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Extract x and y positions of nodes
x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

# Calculate node degrees
node_degrees = dict(G.degree())

# Plot communities based on real position
plt.figure(figsize=(12, 5))

# Plot based on real position
plt.subplot(1, 2, 1)
community_colors = [f"C{idx}" for idx in range(len(community_list))]  # Generate community colors
for source, target in G.edges():
    pos_src = (x_positions[neuron_names == source][0], y_positions[neuron_names == source][0])
    pos_tgt = (x_positions[neuron_names == target][0], y_positions[neuron_names == target][0])
    plt.annotate("",
                 xy=pos_src, xycoords='data',
                 xytext=pos_tgt, textcoords='data',
                 arrowprops=dict(arrowstyle="-", color='gray', alpha=0.3, linewidth=1,
                                 connectionstyle="arc3,rad=0.2", zorder=1))  # Adjust the curvature with rad parameter

for idx, (community, color) in enumerate(zip(community_list, community_colors)):
    community_x = x_positions[np.isin(neuron_names, community)]
    community_y = y_positions[np.isin(neuron_names, community)]
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 10 for deg in community_degrees]  # Adjust the multiplier as needed
    plt.scatter(community_x, community_y, label=f'Community {idx + 1}', s=node_size, c=color, zorder=3)

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Network Community Visualization (Real Position)')
plt.grid(True, color='gray', alpha=0.2, linewidth=0.5)  # Adjust grid lines
plt.axis('equal')  # Ensure aspect ratio is equal
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot based on topological closeness using Kamada-Kawai layout
plt.subplot(1, 2, 2)
pos = nx.kamada_kawai_layout(G)  # Using Kamada-Kawai layout
for idx, (community, color) in enumerate(zip(community_list, community_colors)):
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 10 for deg in community_degrees]  # Adjust the multiplier as needed
    nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=color, node_size=node_size)
for source, target in G.edges():
    pos_src = pos[source]
    pos_tgt = pos[target]
    nx.draw_networkx_edges(G, pos, edgelist=[(source, target)], width=0.5, alpha=0.7, connectionstyle="arc3,rad=0.2", edge_color='gray')  # Curved edges without arrowheads

plt.title('Network Community Visualization (Kamada-Kawai Layout)')

plt.tight_layout()
plt.show()

- Plot_04 _ node color

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt


# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Consider only positive correlations
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Extract x and y positions of nodes
x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

# Calculate node degrees
node_degrees = dict(G.degree())

# Define lighter colors for communities
lighter_community_colors = []
for color in community_colors:
    rgb_color = ColorConverter().to_rgb(color)
    lighter_color = tuple([min(1, c + 0.3) for c in rgb_color])  # Adjust the increment (0.3) as needed
    lighter_community_colors.append(lighter_color)

# Plot communities based on real position
plt.figure(figsize=(12, 5))

# Plot based on real position
plt.subplot(1, 2, 1)
community_colors = [f"C{idx}" for idx in range(len(community_list))]  # Generate community colors
for source, target in G.edges():
    pos_src = (x_positions[neuron_names == source][0], y_positions[neuron_names == source][0])
    pos_tgt = (x_positions[neuron_names == target][0], y_positions[neuron_names == target][0])
    plt.annotate("",
                 xy=pos_src, xycoords='data',
                 xytext=pos_tgt, textcoords='data',
                 arrowprops=dict(arrowstyle="-", color='gray', alpha=0.4, linewidth=1,
                                 connectionstyle="arc3,rad=0.2", zorder=1))  # Adjust the curvature with rad parameter

for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
    community_x = x_positions[np.isin(neuron_names, community)]
    community_y = y_positions[np.isin(neuron_names, community)]
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 15 for deg in community_degrees]  # Adjust the multiplier as needed
    plt.scatter(community_x, community_y, label=f'Community {idx + 1}', s=node_size, c=color, zorder=3)

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Network Community Visualization (Real Position)')
plt.grid(True, color='gray', alpha=0.2, linewidth=0.5)  # Adjust grid lines
plt.axis('equal')  # Ensure aspect ratio is equal
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot based on topological closeness using Kamada-Kawai layout
plt.subplot(1, 2, 2)
pos = nx.kamada_kawai_layout(G)  # Using Kamada-Kawai layout
for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 10 for deg in community_degrees]  # Adjust the multiplier as needed
    nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=color, node_size=node_size, alpha=1)  # Set alpha value for lighter color
for source, target in G.edges():
    pos_src = pos[source]
    pos_tgt = pos[target]
    nx.draw_networkx_edges(G, pos, edgelist=[(source, target)], width=0.5, alpha=0.9, connectionstyle="arc3,rad=0.2", edge_color='gray')  # Curved edges without arrowheads

plt.title('Network Community Visualization (Kamada-Kawai Layout)')

plt.tight_layout()
plt.show()

- Plot Heatmap_ Reaorder neurons based on their community to plot the modular heatmap

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix.index = neuron_names  # Set index names to neuron names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.60:  # Consider only positive correlations
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)

# Community detection using Louvain method
partition = community_louvain.best_partition(G)
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Print communities
for idx, community in enumerate(community_list):
    print(f"Community {idx + 1}: {community}")

# Extract ordered neurons
ordered_neurons = [neuron for community in community_list for neuron in community]

# Create a symmetric correlation matrix with ordered neurons
ordered_data = pd.DataFrame(index=ordered_neurons, columns=ordered_neurons)
for i, neuron_i in enumerate(ordered_neurons):
    for j, neuron_j in enumerate(ordered_neurons):
        correlation_coefficient = correlation_matrix.loc[neuron_i, neuron_j]
        ordered_data.at[neuron_i, neuron_j] = correlation_coefficient
        ordered_data.at[neuron_j, neuron_i] = correlation_coefficient

# Ensure diagonal is zero
np.fill_diagonal(ordered_data.values, 0)

# Convert correlation coefficients to floats
ordered_data = ordered_data.astype(float)

# Plot the heatmap
plt.figure(figsize=(7, 5))

# Create a colormap for community colors
num_communities = len(community_list)
community_colors = sns.color_palette("hsv", num_communities)
community_colormap = ListedColormap(community_colors)

sns.heatmap(ordered_data, cmap='Greys_r', vmin=ordered_data.values.min(), vmax=ordered_data.values.max(),
            linewidths=0.5, linecolor='black', alpha=0.3, cbar=True, square=True, mask=ordered_data.isnull(),
            annot=False, fmt='.2f')

# Draw squares around neurons of each community with respective colors
ax = plt.gca()
start = 0
for idx, community in enumerate(community_list):
    end = start + len(community)
    color = community_colors[idx]
    rect = plt.Rectangle((start, start), end - start, end - start, linewidth=1, edgecolor=color, facecolor=color,alpha=0.5 )
    ax.add_patch(rect)
    start = end

plt.title('Community-Based Correlation Matrix Heatmap')
plt.xlabel('Neurons')
plt.ylabel('Neurons')
plt.show()

- Unified code for community-based topology, geometry maps and  Heat Map 

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import community as community_louvain
import matplotlib.pyplot as plt
from matplotlib.colors import ColorConverter
import seaborn as sns
import random


# Load your data
excel_file = "Thresholded_Matrix_0.6.xlsx"
data = pd.read_excel(excel_file)
neuron_names = data.iloc[:, 0].values
category = data.iloc[:, 4].values

# Sort correlation matrix based on neuron_names order
correlation_matrix = data.iloc[:, 5:].copy()
correlation_matrix.columns = neuron_names
correlation_matrix.index = neuron_names  # Set index names to neuron names
correlation_matrix = correlation_matrix.reindex(neuron_names, axis=1)
correlation_matrix = correlation_matrix.reindex(neuron_names)
#print (correlation_matrix)

# Ensure diagonal is zero
np.fill_diagonal(correlation_matrix.values, 0)

# Create a weighted graph
G = nx.Graph()
for i, neuron_i in enumerate(neuron_names):
    for j, neuron_j in enumerate(neuron_names):
        if i < j:  # To avoid duplicates and self-loops
            correlation_coefficient = correlation_matrix.iloc[i, j]
            if correlation_coefficient > 0.65:  # Consider only positive correlations
                G.add_edge(neuron_i, neuron_j, weight=correlation_coefficient)


                
######################### Extract Communities in each network ###################################            

# Community detection using Louvain method
partition = community_louvain.best_partition(G, resolution=1.0)  # Adjust the resolution parameter as needed
community_list = [[] for _ in range(max(partition.values()) + 1)]
for neuron, community_id in partition.items():
    community_list[community_id].append(neuron)

# Extract x and y positions of nodes
x_positions = data.iloc[:, 1].values  # Assuming x positions are in the second column
y_positions = data.iloc[:, 2].values  # Assuming y positions are in the third column

# Calculate node degrees
node_degrees = dict(G.degree())

# Define community colors
community_colors = [f"C{idx}" for idx in range(len(community_list))]

# Define lighter colors for communities
lighter_community_colors = []
for color in community_colors:
    rgb_color = ColorConverter().to_rgb(color)
    lighter_color = tuple([min(1, c + 0.3) for c in rgb_color])  # Adjust the increment (0.3) as needed
    lighter_community_colors.append(lighter_color)

###################################### Plot Connectivit Maps ######################################   
    
# Plot communities based on real position
plt.figure(figsize=(12, 5))

# Plot based on real position
plt.subplot(1, 2, 1)
community_colors = [f"C{idx}" for idx in range(len(community_list))]  # Generate community colors
for source, target in G.edges():
    pos_src = (x_positions[neuron_names == source][0], y_positions[neuron_names == source][0])
    pos_tgt = (x_positions[neuron_names == target][0], y_positions[neuron_names == target][0])
    plt.annotate("",
                 xy=pos_src, xycoords='data',
                 xytext=pos_tgt, textcoords='data',
                 arrowprops=dict(arrowstyle="-", color='gray', alpha=0.4, linewidth=0.3,
                                 connectionstyle="arc3,rad=0.2", zorder=1))  # Adjust the curvature with rad parameter

for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
    community_x = x_positions[np.isin(neuron_names, community)]
    community_y = y_positions[np.isin(neuron_names, community)]
    
    # Add jitter to x and y positions
    jittered_community_x = [x + random.uniform(-30, 30) for x in community_x]
    jittered_community_y = [y + random.uniform(-30, 30) for y in community_y]
    
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 3 for deg in community_degrees]  # Adjust the multiplier as needed
    plt.scatter(jittered_community_x, jittered_community_y, label=f'Community {idx + 1}',
                s=node_size, c=color, zorder=3, edgecolor='black', linewidth=0.3)

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Network Community Visualization (Real Position)')
plt.grid(True, color='gray', alpha=0.2, linewidth=0.5)  # Adjust grid lines
plt.axis('equal')  # Ensure aspect ratio is equal
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
# Adjust the y-axis limits to -200 and 1600
plt.ylim(-200, 1600)
# Save the plot as PDF
plt.savefig('network_community_visualization_poistionBased.pdf', format='pdf')
# Save the plot as SVG
plt.savefig('network_community_visualization_poistionBased.svg', format='svg')


# Plot based on topological closeness using Kamada-Kawai layout
plt.subplot(1, 2, 2)
pos = nx.kamada_kawai_layout(G, scale=2.0)  # Using Kamada-Kawai layout with increased scale for node separation
for idx, (community, color) in enumerate(zip(community_list, lighter_community_colors)):  # Use lighter colors
    community_degrees = [node_degrees[node] for node in community]
    node_size = [deg * 3 for deg in community_degrees]  # Adjust the multiplier as needed
    
    plt.scatter([pos[node][0] for node in community], [pos[node][1] for node in community], label=f'Community {idx + 1}',
                s=node_size, c=color, zorder=3, edgecolor='black', linewidth=0.3)

for source, target in G.edges():
    pos_src = pos[source]
    pos_tgt = pos[target]
    nx.draw_networkx_edges(G, pos, edgelist=[(source, target)], 
                           width=0.2, alpha=0.9, connectionstyle="arc3,rad=0.2",
                           edge_color='gray')  # Curved edges without arrowheads

plt.title('Network Community Visualization (Kamada-Kawai Layout)')

plt.tight_layout()

# Save the plot as PDF
plt.savefig('network_community_visualization_Topology.pdf', format='pdf')
# Save the plot as SVG
plt.savefig('network_community_visualization_Topology.svg', format='svg')

plt.show()



############################################ Plot community-based Hetmaps #################################

# Print communities
for idx, community in enumerate(community_list):
    print(f"Community {idx + 1}: {community}")

# Extract ordered neurons  (neurons that are within communities will be selected)
ordered_neurons = [neuron for community in community_list for neuron in community]

# Create a symmetric correlation matrix with ordered neurons
# first: generates list of node pairs with their correlation coefficient attributed. 
# Then: it generates a symmetric matrix based on list 
ordered_data = pd.DataFrame(index=ordered_neurons, columns=ordered_neurons)
for i, neuron_i in enumerate(ordered_neurons):
    for j, neuron_j in enumerate(ordered_neurons):
        correlation_coefficient = correlation_matrix.loc[neuron_i, neuron_j]
        ordered_data.at[neuron_i, neuron_j] = correlation_coefficient
        ordered_data.at[neuron_j, neuron_i] = correlation_coefficient


# Ensure diagonal is zero
np.fill_diagonal(ordered_data.values, 0)

# Convert correlation coefficients to floats
ordered_data = ordered_data.astype(float)

# Plot the heatmap
plt.figure(figsize=(7, 5))

# Plot the heatmap with grayscale colormap
sns.heatmap(ordered_data, cmap='Greys', vmin=ordered_data.values.max(), vmax=ordered_data.values.min(),
            linewidths=0.5, linecolor='black', alpha=0.5, cbar=True, square=True, mask=ordered_data.isnull(),
            annot=False, fmt='.2f')

# Draw squares around neurons of each community with respective colors
ax = plt.gca()
start = 0
for idx, community in enumerate(community_list):
    end = start + len(community)
    color = community_colors[idx]
    rect = plt.Rectangle((start, start), end - start, end - start, linewidth=1, edgecolor=color, facecolor=color, alpha=0.5)
    ax.add_patch(rect)
    start = end

plt.title('Community-Based Correlation Matrix Heatmap')
plt.xlabel('Neurons')
plt.ylabel('Neurons')

# Save the plot as PDF
plt.savefig('network_community_visualization_HeatMap.pdf', format='pdf')
# Save the plot as SVG
plt.savefig('network_community_visualization_HeatMap.svg', format='svg')
plt.show()